In [1]:
import sys
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')

train_df = shuffle(train_df)
valid_df = shuffle(valid_df)

print(train_df.head())


                                                  image                   id  \
4364  ../../data/Train/CameraRGB/episode_0006_000095...  episode_0006_000095   
4519  ../../data/Train/CameraRGB/episode_0007_000050...  episode_0007_000050   
4588                 ../../data/Train/CameraRGB/560.png                  560   
4196                 ../../data/Train/CameraRGB/851.png                  851   
3052  ../../data/Train/CameraRGB/episode_0007_000046...  episode_0007_000046   

                                                  label  
4364  ../../data/Train/CameraSeg/episode_0006_000095...  
4519  ../../data/Train/CameraSeg/episode_0007_000050...  
4588                 ../../data/Train/CameraSeg/560.png  
4196                 ../../data/Train/CameraSeg/851.png  
3052  ../../data/Train/CameraSeg/episode_0007_000046...  


In [8]:
from models.fcn8 import model_fcn8

model = model_fcn8(12, image_shape=(320, 416, 3), keep_prob=0.5)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 320, 416, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 320, 416, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 320, 416, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 160, 208, 64) 0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [9]:
from gen.datagen import balanced_generator_from_df

BATCH_SIZE = 32

model_dir = '../../saved_models/fcn8/fcn8_v3/'


train_gen = balanced_generator_from_df(train_df, BATCH_SIZE, (320, 416))
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (320, 416))


In [27]:
from train import euclidean_distance_loss
from keras.utils.training_utils import multi_gpu_model
from keras.optimizers import RMSprop

gpus = 2

# opt = RMSprop(lr=1e-06)
model_gpu = multi_gpu_model(model, gpus)
model_gpu.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])


In [ ]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=m, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                  gpus = 2)

Epoch 1/100
75/75 [==============================] - 88s 1s/step - loss: 0.7549 - acc: 0.9511 - val_loss: 1.7548 - val_acc: 0.8854

Epoch 00001: val_loss improved from inf to 1.75483, saving model to ../../saved_models/fcn8/fcn8_v3//model.hdf5
Epoch 2/100
75/75 [==============================] - 78s 1s/step - loss: 0.6514 - acc: 0.9574 - val_loss: 1.7256 - val_acc: 0.8855

Epoch 00002: val_loss improved from 1.75483 to 1.72558, saving model to ../../saved_models/fcn8/fcn8_v3//model.hdf5
Epoch 3/100
75/75 [==============================] - 79s 1s/step - loss: 0.6150 - acc: 0.9589 - val_loss: 1.6912 - val_acc: 0.8836

Epoch 00003: val_loss improved from 1.72558 to 1.69121, saving model to ../../saved_models/fcn8/fcn8_v3//model.hdf5
Epoch 4/100
75/75 [==============================] - 80s 1s/step - loss: 0.5833 - acc: 0.9579 - val_loss: 1.6953 - val_acc: 0.8783

Epoch 00004: val_loss did not improve
Epoch 5/100
75/75 [==============================] - 79s 1s/step - loss: 0.5406 - acc: 0.9

In [29]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [30]:
model.save('../../saved_models/fcn8/fcn8_v3/model_saved.h5')

In [ ]:
#valid_gen = train_and_lab_gen_func(valid_df, image_size=(600, 800), target_size=(480, 480), batch_size = BATCH_SIZE * gpus)
#test_gen = train_and_lab_gen_func(test_df, image_size=(600, 800),  target_size=(480, 480),  batch_size = BATCH_SIZE * gpus)